# AgentKitでエージェント型ワークフローの構築、デプロイ、最適化を行う

## はじめに

DevDay 2025で、開発者と企業がエージェントを構築、デプロイ、最適化するための完全なツールセットである[AgentKit](https://openai.com/index/introducing-agentkit/)を発表しました。AgentKitは相互接続された構成要素のセットです：

* [Agent Builder](https://platform.openai.com/docs/guides/agents/agent-builder): エージェントワークフローを視覚的に構築・反復する
* [ChatKit](https://platform.openai.com/docs/guides/chatkit): チャットベースのワークフローをアプリに簡単に組み込む
* [Evals](https://platform.openai.com/docs/guides/evals?api-mode=responses): LLMを活用したアプリのパフォーマンスを向上させる

**このクックブックでは、AgentKitを通じてエンドツーエンドの旅を行います - アプリを構築、デプロイ、最適化します。AgentKitの構成要素がどのように連携するかを理解し、エージェントワークフローをより迅速かつ確実に本番環境に導入できるようになります。**

以下のステップを順番に説明します：

1. アプリのバックエンドとして機能するワークフローをAgent Builderで構築する
2. ChatKit webコンポーネントを使用してフロントエンドチャットアプリをデプロイする
3. プロンプト最適化とトレース評価を使ってEvalsでワークフローのパフォーマンスを最適化する

## Agent Builderを使用したマルチエージェントワークフローの構築

Agent Builderを使用して、アプリの基盤となる初期ワークフローを作成することから始めましょう。Agent Builderは、ノードをドラッグアンドドロップしてエージェントワークフローを設計できるビジュアルキャンバスです。Agent Builderについて詳しくは[こちら](https://platform.openai.com/docs/guides/agent-builder)で学ぶことができます。追加機能やサポートされているノードの完全なリストも含まれていますが、このクックブックでは3つのAgentノードが順次動作するシンプルなワークフローを作成します。

キュレーションされた学習推奨を通じて人々のキャリア向上を支援するシンプルなアプリを構築します。ユーザーは履歴書をアップロードして理想の職業を教えてくれれば、開発すべきスキルに基づいてキュレーションされた学習コースセットを受け取ることができます。そこで、3つのエージェントを作成します：

1. **履歴書抽出**エージェント - アップロードされた履歴書を解析し、関連するスキルと経験を抽出
2. **キャリア分析**エージェント - 目標職種に対する知識ギャップを分析
3. **コース推奨**エージェント - 上流の情報を使用して関連するオンラインコースを提案

これらの各エージェントを順次構築していきましょう。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_workflow.png" /></center></div>

### 1. 履歴書抽出エージェント

このエージェントは、アップロードされた履歴書を解析し、下流の分析で使用されるスキルと経験の構造化された出力を返す責任を持ちます。以下のプロンプトを使用します：

```text
入力された履歴書から情報を抽出して要約し、カテゴリ別に出力を整理し、利用可能な場合は文脈を提供してください。
- 提供された入力を分析して、スキルと職歴を特定してください。
- 各スキルまたは経験について、テキストから裏付けとなる文脈や証拠を抽出してください（例：Pythonのスキルの場合、文脈は「[会社名]で3年間データ分析にPythonを使用」など）。
- すべてのスキルと経験が抽出されるまで、テキストの確認を続けてください。
```

このエージェントには`gpt-5`を使用し、`minimal`推論から始めますが、必要に応じて後でモデルを変更することもできます。また、構造化された応答を強制し（出力形式をJSONに選択し、スキーマを追加することで）、モデルに求める正確なデータ形状を返すよう指示します。（この構造化出力のJSONスキーマは[こちら](https://cdn.openai.com/cookbook/agent_walkthrough/Skills_schema.json)で確認できます。）

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_json_schema.png" width="50%"/></center></div>

### 2. キャリア分析エージェント

このエージェントは、個人が望む職業的またはキャリア目標に向けて進歩するために必要なスキルと知識のギャップを分析します。このエージェントには`gpt-5`を使用し、推論努力を`low`に設定します。これにより、このレベルの分析には十分な推論を提供しながら、レスポンスを十分に高速に保つことができます。

```text
Your role is to analyze skill and knowledge gaps for an individual to progress to a desired professional or career goal. 

You will receive a list of the already-obtained skills and experiences of an individual, as well as a description of the goal. First, understand the goal and analyze the critical skills or knowledge areas required for achieving the goal. Then, compare the requirements to what the individual already possesses. 

Return a list of the top 3-5 skills that the individual does not possess, but are important for their professional goal. Along with each skill, include a brief description.

Individual's expressed goal: 
{{workflow.input_as_text}}

Already-obtained skills and experiences:
{{input.output_text}}
```

プロンプトには、{{ブラケット}}で囲まれた前のノードからのコンテキストが含まれていることに注意してください。「コンテキストを追加」をクリックして、モデルへのコンテキスト変数を確認することもできます。

### 3. コース推薦エージェント

このエージェントは、ウェブ検索ツールを使用して、特定されたスキルギャップにマッチするオンライン研修コースを見つけて選択します。`gpt-5`を`minimal`推論で使用し、このエージェントにWeb Searchを装備します。

```text
Your job is to identify and recommend online training courses that help develop one or more of the skills identified. Given the list of required skills and descriptions below, return a list of 3-5 online courses along with course details.

Skills: {{input.output_text}}
```

## ワークフローのテスト

**初期ワークフローを構築したので、Agent Builderのプレビュー機能を使って試してみましょう！** まず、ワークフローを公開します。これにより、名前付きでバージョン管理された（一意のワークフローIDを持つ）コピーが作成され、同僚と共有したり、必要に応じてバージョン間でデプロイや復元を行うことができます。

プレビューでは、Agent Builder内から直接、チャットユーザーと同じようにワークフローと対話できます。履歴書をアップロードし、理想の仕事の説明を入力して送信ボタンをクリックすると、ワークフローが画面左側の各ノードを段階的に進行し、画面右側に各ノードからの出力が表示されます。

例として、学校の教育長になりたいと考えている教師の履歴書をアップロードしました。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_workflow_preview.png" /></center></div>

ワークフローが履歴書解析、スキル分析、ウェブ検索を通じて進行する様子を追跡できます。ワークフローが完了すると、期待通り検索パラメータに合致するオンラインプログラムのリストが表示されます。

**すべて順調です - これでアプリをデプロイする準備が整いました！** 変更を確実に公開し、ワークフローIDを取得しましょう。画面上部の「Code」を選択して、再度IDにアクセスできます。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_code.png" /></center></div>

先ほど構築したワークフローをサポートするコードにアクセスするには、「Agents SDK」タブを使用できることに注意してください。このコードはAgents SDKパッケージ（[JavaScript/TypeScript](https://github.com/openai/openai-agents-js)または[Python](https://github.com/openai/openai-agents-python)）を使用して実装されています。これは、独自の環境でワークフローを実行したい場合や、カスタム機能でさらに開発を進めたい場合に最適なオプションです。（ただし、以下で説明するように、AgentKitを統合的に使用することの利点の一部を逃すことになります。）

## ChatKitを使用したチャットアプリのデプロイ

アプリをデプロイするために、[ChatKitスターターテンプレート](https://github.com/openai/openai-chatkit-starter-app)を使用して、ChatKit webコンポーネントを使ったチャットベースのアプリを立ち上げます。

その前に、AgentKit全体のツールスイートがエージェントワークフローのデプロイに提供する全オプションについて説明する価値があります。Agent Builderでワークフローを構築し、ツール内で直接実行（プレビュー）したり、独自の環境で使用するためにAgents SDKコードとしてエクスポートしたりする方法をすでに見てきました。今度は、Agent BuilderのワークフローIDを使用して、作成したワークフローをバックエンドとして指す独自のフロントエンドに埋め込まれたチャット体験を作成する方法を実演します。（ちなみに、ワークフローバックエンドなしで、ChatKit SDKが提供するリッチなチャットGUIのみを使用することも可能です - 詳細は[こちら](https://platform.openai.com/docs/guides/custom-chatkit)をご覧ください。）

それでは、ChatKitスターターテンプレートを使い始めて、ワークフローを組み込みましょう。スターターテンプレートを使用すると、新しく作成したワークフローを使ったチャットベースのアプリを簡単に立ち上げることができます。リポジトリの[Getting Started](https://github.com/openai/openai-chatkit-starter-app?tab=readme-ov-file#getting-started)の手順に従って、Agent BuilderからのワークフローIDを`.env.local`の`NEXT_PUBLIC_CHATKIT_WORKFLOW_ID`の値として入力し、`npm install`を実行してから`npm run dev`を実行してアプリをテストするだけです。

わずか数分で、ワークフローがフロントエンドのチャットインターフェースに埋め込まれ、稼働します！

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_chat_app.png" /></center></div>

## ワークフローとユーザーエクスペリエンスの迅速な反復

AgentKitの最も価値ある側面の一つは、エージェント型アプリケーションの実験、反復、改善を迅速に行えることです。アプリの機能を向上させ、より豊かなチャット体験を提供するために、いくつかの簡単な変更を加えてみましょう。

**まず、カスタムテーマを追加して**フロントエンドにスタイルを与えながら、ネイティブなチャット体験を維持しましょう。ここで素晴らしいリソースとなるのが[ChatKit Studio](https://chatkit.studio/)です。これには、ChatKitのカスタマイズオプションを探索するためのプレイグラウンド、Widget Builder（これについてはすぐに実際の動作を見ることができます）、そしてインスピレーションのためのサンプルアプリとギャラリーが含まれています。カスタムテーマを取得するために、ChatKitの[Playground](https://chatkit.studio/playground)を使用して視覚的に希望するスタイルオプションを選択し、プレイグラウンド画面上部の`</>`アイコンをクリックして設定コードを取得します。

このコードの`theme`オブジェクトを使用して、[lib/config.ts](https://github.com/openai/openai-chatkit-starter-app/blob/main/lib/config.ts)にあるデフォルトテーマを上書きします。そのファイルを編集している間に、スターター プロンプト、挨拶テキスト、プレースホルダーのコピーもより適切な値に調整します：

```text
export const GREETING = "Upload your resume, and tell me the job you're looking to get!";
export const PLACEHOLDER_INPUT = "Describe your dream job, and don't forget to attach your resume!"; 
```

**次に、カスタムウィジェットを設計して**推奨コースをより直感的な形式で表示し、ユーザーが理解しやすく比較しやすくします。[Widget Builder](https://widgets.chatkit.studio/)を使用して、求めている出力を簡単に記述し、さらに編集できるLLM生成の出発点を取得できます。この例では、コースを清潔で構造化された形式のリストで表示し、推奨事項の下に要約も表示します。Widget Builderでは、ウィジェットコード（左上）だけでなく、サンプルデータ（左下）と、そのデータがエンドユーザー向けにウィジェット内でどのようにレンダリングされるか（右）も確認できます。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_widget_builder.png" /></center></div>

デザインに満足したら、.widgetファイルをダウンロードできます。（以下のスクリーンショットで使用されているファイルは[こちら](https://cdn.openai.com/cookbook/agent_walkthrough/Course%20recommendation.widget)にあります。）

設計したカスタムウィジェットをチャットアプリで実際に使用するには、**推薦エージェントのレスポンスの一部としてウィジェットコンポーネントを返すようにワークフローに指示する必要があります**。そこで、`Course recommendations`エージェントに戻り、出力形式として`Widget`を選択し、.widgetファイルをアップロードします。

エージェントは、ウィジェットに入力するために必要なJSON形式を自動的に出力する方法を理解します。ただし、各コースについてモデルが取得する必要がある情報をより正確に指示するために、エージェントプロンプトを更新する必要があります。

```text
Your job is to identify and recommend online training courses that help develop one or more of the skills identified. Given the list of required skills, return a list of 3-5 online courses along with course details including course name, provider (school or program), recommendation reason (a brief sentence on why you're recommending the course), course format, and URL. In addition to the list of courses, share a few-sentence summary of the recommendations you're making. 
```

最後に、履歴書を扱っているため、個人識別情報（PII）が適切でない場所に伝播されないようにするためのガードレールをワークフローに追加します。このガードレールを履歴書パーサーとキャリア分析エージェントの間に挿入することで、履歴書エージェントの下流にあるものが名前や連絡先情報などのPIIにアクセスすることを防ぐことができます。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_guardrails.png" /></center></div>

## プロンプト最適化とトレース評価を使用したシステムパフォーマンスの向上

次に、Evalsとのネイティブ統合がどのように個々のエージェントとワークフロー全体の両方を簡単に最適化するのに役立つかを見ていきます。私たちのチャットアプリがデプロイされ（おそらく初期の内部ユーザーやベータテスターに）、システムとやり取りするユーザーの実際の例がいくつかあると想像してみましょう。

このアプリケーションがパフォーマンスと品質が重要な本格的なシステムに開発される場合、開発プロセスにより早期かつ体系的にevalsを組み込みたいと考えるでしょう。（初期evalsセットと確立された正解データの構築、evalsとビジネス指標のマッピング、ビジネス目標を推進するためのシステムの段階的改善について詳しくは、[Eval Driven System Design](https://cookbook.openai.com/examples/partners/eval_driven_system_design/receipt_inspection)で学ぶことができます。）

しかし、このクックブックでは、LLMアプリ開発の一部としてのevalsの背後にある技術にはあまり焦点を当てず、AgentKitがこれらの技術をより簡単に実装できる方法により重点を置きます。パフォーマンスの向上を2つの方法で推進します：まず、プロンプト最適化ツールを使用して**ワークフロー内の単一エージェントノードを最適化**し、次にトレース評価を使用して**ワークフロー全体を最適化**します。

### 単一エージェントの最適化

Course recommendationsエージェントを詳しく調べて、ユーザーへの推奨の品質を向上させることができるかを確認したいと思います。テスト環境からこのエージェントのサンプルプロンプトを分離しました。（[APIプラットフォームのLogsタブ](https://platform.openai.com/logs?api=responses)でも完了したものにアクセスできます。このクックブックの例では、使用するデータセットに[こちら](https://cdn.openai.com/cookbook/agent_walkthrough/course_recommendations_dataset.csv)からアクセスできます。）

Agent Builderから直接エージェントの最適化を開始できます。Course recommendationsエージェントを選択し、エージェントモーダルの右下にある「Evaluate」をクリックします。これにより、Evals内の**Datasets**機能に直接移動します。

エージェントの設定がコピーされ、最適化の準備が整ったことがわかります。まず、サンプルプロンプトを含むデータファイルをアップロードし（列名は入力変数と出力変数と一致する必要があることに注意）、「Generate output」をクリックして応答を生成しましょう。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_generate_output.png" /></center></div>

それでは、**人間による注釈**と**モデル評価者**を作成しましょう。「Columns」を選択して、評価（サムズアップ/ダウン）とフィードバック（テキスト入力）を追加し、サンプルを手動でレビューして、これらのフィールドに高品質なフィードバックを入力します。また、指定した基準に基づいてエージェントの出力を自動的に評価するモデル評価者もいくつか追加します。この例では、コース推薦が特定されたスキルギャップに関連しているか（関連性）、すべてのスキルギャップが対処されているか（網羅性）、提示される推薦要約が適切か（スタイル）について懸念があるかもしれません。

各基準に対するモデル評価者プロンプトの例は以下の通りです：

```text
[relevance] You are evaluating whether a list of recommended courses is relevant to the skills described. Return a pass if all courses are relevant to at least one skill, and fail otherwise.

[coverage] You are evaluating whether a list of recommended courses covers all of the skills described. Return a pass if all of the skills are covered by at least one course, and fail otherwise.

[summary] You are evaluating whether the summary recommendation provided is relevant, thoughtful, and related to the recommended courses proposed. Evaluate the summary recommendation on a scale of 0 to 1, with 1 being the highest quality. 
```

モデル評価者にはGPT-5を使用し、要約評価者には0.7の閾値を設定します。

次に、Grade > All gradersを選択して、これらの評価者をシステム出力に対して実行します。評価が進行すると、各例がモデル評価者の基準でどのようにスコアされたかを示すセルが入力されるのが見え始めます。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_grading.png" /></center></div>

**ここで魔法が起こります：私たちが提供したフィードバック（モデル評価者の出力と人間が提供したフィードバックの両方）に基づいて、プロンプトを自動的に書き直すためにOptimizeをクリックできます。**

新しいプロンプトを確認すると、コースの説明をより具体的にし、異なるスキルのより良いカバレッジを目指すようにモデルに指示する新しい**Requirements**と**Output**フォーマットセクションがプロンプトに含まれていることがわかります。

```text
Requirements:

- Use the web search tool to find and verify real, currently available online courses and their direct URLs.
- Return 3–5 courses that collectively cover the skills. If a course spans multiple skills, indicate it. 
- Be specific and concise. Each course description must be one sentence (max 35 words) focused on outcomes and topics tied to the skills. 
- Provide plain text only; no citations or references.

Output format:

1) Summary (2–4 sentences) explaining how the selections address the skills and any coverage tradeoffs. 
2) Courses (3–5 items). For each course, include exactly:` 
   - Course name — Provider (school or program)
   - Description: [one sentence, max 35 words]  
   - URL: [direct course page]
   - Skills covered: [list skill names from below]
```

これで、Updateをクリックして、Agent Builderのワークフローに新しいプロンプトを自動的に挿入できます。**わずか数分で、実際のプロンプト例とフィードバックを使用してシステムのパフォーマンスを自動的に改善することができました - すべてAgent BuilderとEvalsツール内で直接行えます。**

（このクックブックでは、この評価出力を使用してプロンプトを自動的に最適化しましたが、モデルがどのような方法で間違いを犯している可能性があるかを理解するために、特定の失敗例を検証することは非常に有用です。この分析は、より正確な人間ベースまたはモデル評価フィードバックを生成し、特定の失敗モードに対するパフォーマンス向上に役立つデータを合成するのに役立ちます。）

### ワークフロー全体の最適化

各エージェントノードの個別のパフォーマンスに満足したら、次にワークフロー全体に注意を向けることができます。

これまで、複数のエージェントが関わる複雑なワークフローのパフォーマンスを分析・改善するためには、システムがどこで、なぜ失敗しているのかを正確に理解するために、ワークフローのトレース全体を読み通す必要がありました。これは時間のかかるプロセスであり、特に大量のトレース例がある場合には特にそうです。

**トレースグレーディング**を使用することで、自動化されたモデルグレーディングを使用してトレースの完全なセットのエンドツーエンド評価を実行できるようになりました。修正したい動作を記述するグレーダーを作成し、データセット全体でこれらのグレーダーを自動的に実行します。

開始するには、Agent Builderのワークフローに戻り、画面上部の「Evaluate」をクリックします。これにより、ワークフロー実行のために生成されたトレースを詳しく調べることができ、すべてのノードの入力と出力（この場合、各エージェントのプロンプトと応答）を検査することが含まれます。

**複数のエージェントにまたがる**エンドツーエンドワークフローの評価基準を定義して、モデルトレース全体で実行するグレーダーを作成できます。例えば、最終的な推奨要約（エージェント3の出力）がユーザーのキャリア目標に関する初期入力（エージェント1への入力）に関連していることを確認したい場合があります。また、推奨されるコース（エージェント3の出力）がユーザーが既に持っているスキル（エージェント1の出力）と重複していないことを確認したい場合もあります。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_traces.png" /></center></div>

条件文やwhileループを含むワークフローがある場合、より複雑な複数ステップの動作に対して評価を行うことができます。例えば、サポートエージェントは上司にエスカレーションすることなく、ユーザーと3回以上やり取りしてはいけない、といった評価が可能です。

評価基準の完全なセットが揃ったら、「Grade all」を選択してトレースを評価します。このアクションにより「Evaluations」タブに移動し、新しい評価が作成され、評価実行が開始されたことを確認できます。

<div><center><img src="https://cdn.openai.com/cookbook/agent_walkthrough/agentkit_walkthrough_trace_evals.png" /></center></div>

その後、失敗ケースのワークフロートレースを詳しく調べることで、特定のワークフロー実行がなぜ失敗したのか、そして失敗を回避するためにシステムをどのように改善できるかをより深く理解することができます。このアプローチは、失敗モードを反復的に特定し、システムのパフォーマンスを評価し、パフォーマンス向上を目的とした改善に焦点を当てることで、複雑なワークフローをより効率的に最適化するのに役立ちます。

## まとめとリソース

**Agent Builder**、**ChatKit**、**Evals**がどのように連携して、エージェント型ワークフローの構築、デプロイ、最適化を支援するかを実演しました。

具体的な例として、履歴書を分析し、スキルギャップを特定し、オンラインコースを推奨するキャリア開発アプリを通じて、Agent Builderがマルチエージェントワークフローの設計と構築を簡単にし、ChatKitがそれらのワークフローを豊富でカスタマイズ可能なチャットUIに組み込むことを可能にし、Evalsが実際のデータに対するプロンプト最適化とトレース評価を有効にしてループを完成させることを確認しました。

詳細については、このクックブックで言及されたリソースの一覧をご覧ください：

* [Agent Builder ドキュメント](#)
* [ChatKit スターターテンプレート](#)
* [ChatKit Studio](#)
* [Agents SDK](#)
* [Evals](#)

楽しい開発を！